In [17]:
import copy
import re

In [111]:
class Worker:
    def __init__(self, n):
        self.n = n
        self.task = None
        self.timedone = 0

    def __repr__(self):
        return "Worker(n=%d task=%s timedone=%s)" % (self.n, self.task, self.timedone)

class NodeList:
    pattern = re.compile(r"""Step (.+) must be finished before step (.+) can begin.""")
    
    def __init__(self):
        self.parents  = dict()
        self.children = dict()
    
    def add_constraint(self, p, c):
        if p not in self.children:
            self.children[p] = set()
        if c not in self.children:
            self.children[c] = set()
        self.children[p].add(c)
        
        if c not in self.parents:
            self.parents[c] = set()
        if p not in self.parents:
            self.parents[p] = set()
        self.parents[c].add(p)
    
    def pprint(self):
        print("children")
        for p, c in self.children.items():
            print("%s -> %s" % (p, ",".join(sorted(c))))
        print("parents")
        for c, p in self.parents.items():
            print("%s -> %s" % (c, ",".join(sorted(p))))
    
    def graphviz(self):
        print("digraph G {")
        for p, children in self.children.items():
            for c in children:
                print("%s -> %s" % (p, c))
        print("}")
    
    def head_nodes(self):
        head_nodes = []
        for c, p in self.parents.items():
            if not p:
                head_nodes.append(c)
        return head_nodes
    
    def resolve(self):
        parents = copy.deepcopy(self.parents)
        visited = set()
        visitedorder = []
        nodes = self.head_nodes()
        
        while nodes:
            # always grab first in alphabetical order among nodes
            nodes.sort()
            nodes.reverse()
            node = nodes.pop()
            
            # add to visited
            visited.add(node)
            visitedorder.append(node)
            
            # clear node from parent lists
            for i in parents:
                if node in parents[i]: 
                    parents[i].remove(node)
                    
            # add children of current that are available to nodes
            for c in self.children[node]:
                if not parents[c] and c not in visited:
                    nodes.append(c)
                    
        # no more nodes, return result
        return "".join(visitedorder)
    
    def workerstate(self, workers):
        out = []
        for w in workers:
            if w.task == None:
                out.append(".")
            else:
                out.append(w.task)
        return "".join(out)

    def process(self, nwork, delay=0, verbose=False):
        t = -1
        parents = copy.deepcopy(self.parents)
        visited = set()
        visitedorder = []
        avail = self.head_nodes()
        workers = []
        working = set()
        for i in range(nwork):
            workers.append(Worker(i))
        
        while avail or working:
            t += 1
            # clear finished workers first
            for w in workers:
                if t >= timedone and w.task != None:
                    visited.add(w.task)
                    visitedorder.append(w.task)
                    working.remove(w.task)
                        
                    for i in parents:
                        if w.task in parents[i]: 
                            parents[i].remove(w.task)
                                          
                    for c in self.children[w.task]:
                        if not parents[c] and c not in visited:
                            avail.append(c)
                                
                    w.task = None
                    w.timedone = 0
                
            for w in workers:
                if t >= w.timedone:                    
                    if avail:
                        avail.sort()
                        avail.reverse()
                        node = avail.pop()
            
                        # add to working
                        working.add(node)
                        w.task = node
                        w.timedone = t + ord(node) - ord('A') + 1 + delay
            if verbose:
                print("%4d %s %s" % (t, self.workerstate(workers), "".join(visitedorder)))

        return t
    
    @staticmethod
    def from_file(filename):
        nl = NodeList()
        with open(filename, "r") as fh:
            for l in fh:
                #print(l)
                m = NodeList.pattern.match(l.rstrip())
                if m:
                    nl.add_constraint(m.group(1), m.group(2))
        return nl

test = NodeList.from_file("test.txt")
#test.resolve()
test.process(2, verbose=True)

NameError: name 'timedone' is not defined

In [110]:
input = NodeList.from_file("input.txt")
#input.resolve()
input.process(5, delay=60, verbose=True)

   0 GOS.. 
   1 GOS.. 
   2 GOS.. 
   3 GOS.. 
   4 GOS.. 
   5 GOS.. 
   6 GOS.. 
   7 GOS.. 
   8 GOS.. 
   9 GOS.. 
  10 GOS.. 
  11 GOS.. 
  12 GOS.. 
  13 GOS.. 
  14 GOS.. 
  15 GOS.. 
  16 GOS.. 
  17 GOS.. 
  18 GOS.. 
  19 GOS.. 
  20 GOS.. 
  21 GOS.. 
  22 GOS.. 
  23 GOS.. 
  24 GOS.. 
  25 GOS.. 
  26 GOS.. 
  27 GOS.. 
  28 GOS.. 
  29 GOS.. 
  30 GOS.. 
  31 GOS.. 
  32 GOS.. 
  33 GOS.. 
  34 GOS.. 
  35 GOS.. 
  36 GOS.. 
  37 GOS.. 
  38 GOS.. 
  39 GOS.. 
  40 GOS.. 
  41 GOS.. 
  42 GOS.. 
  43 GOS.. 
  44 GOS.. 
  45 GOS.. 
  46 GOS.. 
  47 GOS.. 
  48 GOS.. 
  49 GOS.. 
  50 GOS.. 
  51 GOS.. 
  52 GOS.. 
  53 GOS.. 
  54 GOS.. 
  55 GOS.. 
  56 GOS.. 
  57 GOS.. 
  58 GOS.. 
  59 GOS.. 
  60 GOS.. 
  61 GOS.. 
  62 GOS.. 
  63 GOS.. 
  64 GOS.. 
  65 GOS.. 
  66 GOS.. 
  67 DOSUX G
  68 DOSUX G
  69 DOSUX G
  70 DOSUX G
  71 DOSUX G
  72 DOSUX G
  73 DOSUX G
  74 DOSUX G
  75 D.SUX GO
  76 D.SUX GO
  77 D.SUX GO
  78 D.SUX GO
  79 D..UX GOS
  80 D..UX GOS
  81 D

1025

In [80]:
'GDHUXACIWMOTPYJSRNLEVQFZBK'

'GDHUXACIWMOTPYJSRNLEVQFZBK'